# ESGrader tool

Before running this script, ensure that all of the data from "Data to be Imported for Running Script" folder is imported into the virtual environment to be referenced

In [6]:
from esg_recommender import ESGRecommendationEngine, ProjectType
from spatial_EJ_assessment import process_ejscreen, top_social_variables, top_environmental_variables, ej_indexes

import pandas as pd

# Adjust the below lines to ensure the input data location is correct

In [7]:
#Inputs (adjust)
shapefile_path = "/content/EJScreen.shp"
cejst_shapefile_path = "/content/CEJST.shp"

proj_loc = "Newark, NJ"
survey_data = pd.read_csv('/content/Example Survey Results.csv')
proj_type = ProjectType.SOLAR

In [8]:
# Call the process_ejscreen function
results_df, cejst_check = process_ejscreen(proj_loc, shapefile_path, cejst_shapefile_path)

# Extract top 3 social and environmental variables
top_social = top_social_variables(results_df)
top_environmental = top_environmental_variables(results_df)

# Record highest social and environemntal stressor percentile values for location
location_data = ej_indexes(results_df)

In [9]:
# Generate recommendations

engine = ESGRecommendationEngine()

# Calculate scores based on the ESG weighting
base_scores = engine.calculate_base_scores(survey_data, proj_type)

# Calculate scores based on the impact weighting
impact_scores = engine.calculate_impact_scores(survey_data, proj_type)

# Adjust ESG scores based on the type of project (#Ex. Wind and Solar improve the ENV score by 1.1 times, and also by 1.2 times for job creation)
modified_scores = engine.apply_project_modifiers(base_scores, proj_type)

# Adjust ESG scores based on the EJ context
contextual_scores = engine.incorporate_location_context(modified_scores, location_data)

# Scores data
scores_data = {
        "Contextual ESG Score": contextual_scores,
        "Impact Score": impact_scores
        }

# Create a DataFrame
scores_df = pd.DataFrame(scores_data)

# Fill NaN values with 0 for consistency
scores_df = scores_df.fillna(0)

# Print the DataFrame in tabular form
print("\nEnhanced ESG Scores Table:")
print(scores_df.to_string(index=True, float_format="{:.2f}".format))




Enhanced ESG Scores Table:
                           Contextual ESG Score  Impact Score
environmental                             61.85          0.00
social                                    40.02          0.00
governance                                15.00          0.00
climate_change_mitigation                  0.00         20.00
local_environmental                        0.00         15.00
community_benefits                         0.00         21.67
health_safety                              0.00         10.00
innovation                                 0.00          0.00


In [10]:
recommendations = engine.generate_recommendations(contextual_scores, impact_scores, proj_type)

# Print recommendations
for rec in recommendations:
  print(f"\nRecommendation: {rec.title}")
  print("-" * 50)  # Optional line separator for clarity
  print(f"Expected Impact: ESG +{rec.impact.esg}, Score +{rec.impact.score}")
  print(f"Recommended partners:")
  for partner in rec.partners:
    print(f"\nName: {partner.name}")
    print(f"Type: {partner.type}")
    print(f"Description: {partner.description}")
    print("-" * 50)  # Optional line separator for clarity


  # Print results
print("ADDITIONAL COMMUNITY CONSIDERATIONS:")

print("Top Social Factors to Consider in this Community:")
print(top_social)

print("\nTop Environmental Factors to Consider in this Community:")
print(top_environmental)

print(f"\nEligible for Justice40 funding based on project location? {'YES' if cejst_check else 'NO'}")


Recommendation: Strengthen Governance and Safety Frameworks
--------------------------------------------------
Expected Impact: ESG +18, Score +22
Recommended partners:

Name: SafetyFirst Compliance
Type: certifier
Description: Specialists in health and safety certifications and risk management.
--------------------------------------------------

Name: GovernWell Consulting
Type: consulting
Description: Advisors in corporate governance and ethical practices.
--------------------------------------------------

Recommendation: Enhance Community Engagement Initiatives
--------------------------------------------------
Expected Impact: ESG +20, Score +25
Recommended partners:

Name: Community Builders Network
Type: nonprofit
Description: Organization dedicated to fostering community development and engagement.
--------------------------------------------------

Name: Social Equity Innovations
Type: consulting
Description: Experts in designing and implementing equitable social programs.
--